In [1]:
#import the packet
using Plots
using ProgressBars
using Flux
using Flux: onehot
using Flux: Data.DataLoader
using Flux: onehotbatch, onecold, crossentropy
using Flux: @epochs
using Statistics
using MLDatasets
using CSV
using DataFrames
using Random:shuffle
using EvalMetrics
using Distributions

In [2]:
#parameters:
Batch_size = 12
#there will be 100 models
Pop_size = 100 
#iteration times
iteration = 100

100

In [3]:
#import the dataset
#do 2-class classification:
#decide the wine is red or white
df_red = CSV.read("winequality-red.csv",DataFrame)
df_white = CSV.read("winequality-white.csv",DataFrame)

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide
,Float64,Float64,Float64,Float64,Float64,Float64
1,7.0,0.27,0.36,20.7,0.045,45.0
2,6.3,0.3,0.34,1.6,0.049,14.0
3,8.1,0.28,0.4,6.9,0.05,30.0
4,7.2,0.23,0.32,8.5,0.058,47.0
5,7.2,0.23,0.32,8.5,0.058,47.0
6,8.1,0.28,0.4,6.9,0.05,30.0
7,6.2,0.32,0.16,7.0,0.045,30.0
8,7.0,0.27,0.36,20.7,0.045,45.0
9,6.3,0.3,0.34,1.6,0.049,14.0


In [4]:
#build the model using sigmoid as Activation Function
#The input dim is 12 as 12 features and the output dim is 2 represent A and B
function sigmoidmodel(input_dim)
    model = Chain(
        Dense(input_dim,16,relu),
        Dense(16,64,relu),
        Dense(64,32,relu),
        Dense(32,2),
        softmax
    )
    return model
end

sigmoidmodel (generic function with 1 method)

In [5]:
#Read In the data from the dataframe
csv_mat = [[r.fixedacidity,r.volatileacidity,r.citricacid,r.residualsugar,r.chlorides,r.freesulfurdioxide
        ,r.totalsulfurdioxide,r.density,r.pH,r.sulphates,r.alcohol,r.quality] for r in eachrow(df_red)]
x_redwines = rand(size(csv_mat)[1],12)
for i in 1:size(csv_mat)[1]
    x_redwines[i,1]=csv_mat[i][1]
    x_redwines[i,2]=csv_mat[i][2]
end

csv_mat = [[r.fixedacidity,r.volatileacidity,r.citricacid,r.residualsugar,r.chlorides,r.freesulfurdioxide
        ,r.totalsulfurdioxide,r.density,r.pH,r.sulphates,r.alcohol,r.quality] for r in eachrow(df_white)]
x_whitewines = rand(size(csv_mat)[1],12)
for i in 1:size(csv_mat)[1]
    x_whitewines[i,1]=csv_mat[i][1]
    x_whitewines[i,2]=csv_mat[i][2]
end

In [6]:
#get the matrix
xs = [x_redwines;x_whitewines]
xs = transpose(xs)
ys = [fill(0,size(x_redwines)[1]);fill(1,size(x_whitewines)[1])]
#trun label into 0:1 onthot-bat
ys = Flux.onehotbatch(ys,0:1)
ys = Int.(ys)

2×3198 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [7]:
#shuffle the data randomly
datalength = size(xs)[2]
index = shuffle(collect(1:datalength))

xs = xs[:,index]
ys = ys[:,index]

#split the train data and test data
train_data = Flux.DataLoader((xs[:,1:1000],ys[:,1:1000]),batchsize=20,shuffle=true)
test_data = Flux.DataLoader((xs[:,1000:end],ys[:,1000:end]),batchsize=20,shuffle=true)

DataLoader{Tuple{Matrix{Float64}, Matrix{Int64}}, Random._GLOBAL_RNG}(([6.4 6.8 … 6.8 8.6; 0.595 0.25 … 0.19 0.2; … ; 0.4687831178013747 0.8140574985267094 … 0.335552603556837 0.03321845872278617; 0.8231076583230601 0.9469178335927186 … 0.7160569794491978 0.8490811024457356], [0 0 … 0 0; 1 1 … 1 1]), 20, 2199, true, 2199, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199], true, Random._GLOBAL_RNG())

In [8]:
#Accurate Function
function acc(x,y)
    x = transpose(x)
    y = transpose(y)
    p=[i[2] for i in argmax(x,dims=2)]
    l=[i[2] for i in argmax(y,dims=2)]
    p = reshape(p,size(p)[1])
    l = reshape(l,size(l)[1])
    correct=sum(p.==1)
    len=size(x)[1]
    acc=correct/len
    return acc
end

acc (generic function with 1 method)

In [9]:
#generate population
function generatePop(size)
    population = []
    for i in 1:size
        smodel = sigmoidmodel(Batch_size)
        append!(population,[smodel])
    end
    return population
end

generatePop (generic function with 1 method)

In [10]:
#train models and get the accuracy list
function train_model(model)
    model_acc_list = []
    for (x,y) in train_data
        pred = model(x)
        ac = acc(pred,y)
        append!(model_acc_list,ac)
    end
    meanac = mean(model_acc_list)
    return meanac
end

train_model (generic function with 1 method)

In [11]:
#train models and get the accuracy list
function test_model(model)
    model_acc_list = []
    for (x,y) in test_data
        pred = model(x)
        ac = acc(pred,y)
        append!(model_acc_list,ac)
    end
    meanac = mean(model_acc_list)
    return meanac
end

test_model (generic function with 1 method)

In [12]:
#parameters do add
function parameter_absract(a,b)
    result=Flux.params([i-j for (i,j) in zip(a,b)])
    return result
end

parameter_absract (generic function with 1 method)

In [13]:
#parameters do substract
function parameter_add(a,b)
    result=Flux.params([i+j for (i,j) in zip(a,b)])
    return result
end

parameter_add (generic function with 1 method)

In [14]:
#do iteration here
model_train_mean_acc = []
model_test_mean_acc = []
model_list = generatePop(Pop_size)

gbestTrainList = zeros(Pop_size)
gbestTestList = zeros(Pop_size)

gbestTrainparas = []
gbestTestParas = []

for i in 1:Pop_size
    append!(gbestTrainparas,[Flux.params(model_list[i])])
    append!(gbestTestParas,[Flux.params(model_list[i])])
end

#get the copy of parameters
velocit_train=deepcopy(gbestTrainparas)
velocit_test=deepcopy(gbestTestParas)

for i in 1:iteration
    model_train_acc = []
    model_test_acc = []
    
    pbestTrainList = []
    pbestTestList = []
    
    #train models with gbest and pbest
    for j in 1:Pop_size
        trainac = train_model(model_list[j])
        testac = test_model(model_list[j])
        if gbestTrainList[j] < trainac
            gbestTrainList[j] = trainac
            gbestTrainparas[j] = Flux.params(model_list[j])
        end
        
        if gbestTestList[j] < testac
            gbestTestList[j] = testac
            gbestTestParas[j] = Flux.params(model_list[j])
        end
        #get max
        append!(model_train_acc,trainac)
        append!(model_test_acc,testac)
        append!(pbestTrainList,trainac)
        append!(pbestTestList,testac)
    end
    append!(model_train_mean_acc,mean(model_train_acc))
    append!(model_test_mean_acc,mean(model_test_acc))
    
    #get max pbest and gbest
    pTrainBest = 0
    pTestBest = 0
    pTrainBestparas = Flux.params(model_list[1])
    pTestBestparas = Flux.params(model_list[1])
    
    for k in 1:Pop_size
        if pbestTrainList[k] > pTrainBest
            pTrainBest = pbestTrainList[k]
            pTrainBestparas = Flux.params(model_list[k])
        end
    end
    
    for k in 1:Pop_size
        if pbestTestList[k] > pTestBest
            pTestBest = pbestTestList[k]
            pTestBestparas = Flux.params(model_list[k])
        end
    end
    #do PSO and update the parameters
    for (id,m) in enumerate(model_list)
        #train
        trainlocalup=parameter_absract(gbestTrainparas[id],Flux.params(m))
        trainglobalup=parameter_absract(pTrainBestparas,Flux.params(m))
        trainvup=parameter_add(trainlocalup,trainglobalup)
        velocit_train[id]=parameter_add(velocit_train[id],trainvup)
        velocit_train[id]=parameter_add(Flux.params(m),velocit_train[id])
        
        #test
        testlocalup=parameter_absract(gbestTestParas[id],Flux.params(m))
        testglobalup=parameter_absract(pTestBestparas,Flux.params(m))
        testvup=parameter_add(testlocalup,testglobalup)
        velocit_test[id]=parameter_add(velocit_test[id],testvup)
        velocit_test[id]=parameter_add(Flux.params(m),velocit_test[id])
    end
end

In [15]:
#draw graph
model_train_mean_acc = round.(model_train_mean_acc,digits=3)
model_test_mean_acc = round.(model_test_mean_acc,digits=3)
#print(model_train_mean_acc)
#print(model_test_mean_acc)
x = 1:iteration
x = [log(i) for i in x]
y = hcat(model_train_mean_acc,model_test_mean_acc)
plot(x,y,title="accuracy",label=["Traning" "Testing"])

Any[0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.5513600000000001, 0.55136000000000

In [ ]:
#confusion matrix
#confusion matrix
#set the matrix
confusion = zeros(2,2)
for (x,y) in test_data
    pred = model_list[1](x)
    pred = transpose(pred)
    label= transpose(y)
    
    p = [i[2] for i in argmax(pred,dims=2)]
    l = [i[2] for i in argmax(label,dims=2)]
    
    for (i,j) in zip(l,p)
        confusion[i,j]+=1
    end
end

In [ ]:
#output the confusion matrix
print(confusion)